# ANN Basic

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import numpy as np
import torch
import torch.nn as nn

''' NN
    pytorch에서 Neural Net을 디자인하는 모듈
'''

from sklearn.metrics import accuracy_score

In [3]:
def load_dataset(file, device):
    ''' device
        'cuda', 혹은 'cpu'가 넘어오게 됨
        -> 'cuda'가 넘어오게 되면 gpu(graphic processing unit)에서 실행
        -> 그래픽 내부에서는 주로 행렬 연산이 이루어지기 때문에 gpu를 사용하게 되면 연산이 빨라지고 학습이 빨라짐
    '''

    data = np.loadtxt(file)
    print("DATA = ", data)

    ''' np.loadtxt
        np.array의 형태로 data를 읽어들임
    '''

    # feature, label 분리
    input_features = data[:, :-1]
    print("input features = ", input_features)

    labels = np.reshape(data[:, -1], (4, 1))
    print("labels = ", labels)

    ''' label 형성 과정
        1) data[:, -1] : 1행 4열의 구조, 즉 우리가 알던 행벡터의 transpose 구조가 됨
        2) np.reshape를 통해서 4행 1열의 구조로 바꾸고자 함
    '''

    input_features = torch.tensor(input_features, dtype=torch.float).to(device)
    labels = torch.tensor(labels, dtype=torch.float).to(device)

    ''' torch.tensor
        pytorch가 내부적으로 쓰는 자료구조는 array가 아닌 tensor 형태임
        tensor는 쉽게 생각해서 차원이 큰 array라고 생각하면 될 듯
    '''

    ''' torch.tensor.to(device)
        device는 위에서 얘기했던 cpu를 사용할거냐, gpu(cuda)를 사용할거냐를 나타내는 놈
        이 때, to(device)를 사용해 tensor를 'device'에 올리는 것임
    '''

    return (input_features, labels) # 여기서 input_features 및 labels는 tensor임에 주의!

# tensor는 보기 힘드니까 list의 형태로 보고 싶어!
def tensor2list(input_tensor):
    return input_tensor.cpu().detach().numpy().tolist()

In [4]:
if torch.cuda.is_available():
    device = 'cuda'
else:
    device = 'cpu'

''' torch.cuda.is_available()
    gpu 사용 가능 여부 check
'''

# load dataset
input_features, labels = load_dataset("/content/drive/MyDrive/연구실/기계학습/train.txt", device)

# NN model 디자인
model = nn.Sequential(
    nn.Linear(2, 2, bias=True), nn.Sigmoid(),
    nn.Linear(2, 1, bias=True), nn.Sigmoid()).to(device)

''' nn.Linear(m, n, bias=True)
    m*n 크기의 가중치를 바탕으로 linear하게 연결된 구조
'''

''' nn.Sigmoid()
    nn.Linear를 타고온 값에 sigmoid 함수를 씌우는 역할
'''

# binary classification : cross entropy cost function
loss_func = torch.nn.BCELoss().to(device)

# optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=1)

''' optimizer
    -> 학습하는 함수, 즉 역전파 알고리즘을 뭘로 사용할 것인가?

    torch.optim.SGD(model.parameters(), lr=1)
    - SGD : 위 optimizer는 gradient descent 방법을 사용하는데, 그 중에서도 SGD(Stochastic Gradient Descent) 방법을 사용한다고 보면 됨
    - model.paramters() : optimizie 할 대상 (위 예시에서는 가중치행렬인 W, 편향인 b가 모델의 파라미터가 된다)
    - lr : learning rate
'''

DATA =  [[0. 0. 0.]
 [0. 1. 1.]
 [1. 0. 1.]
 [1. 1. 0.]]
input features =  [[0. 0.]
 [0. 1.]
 [1. 0.]
 [1. 1.]]
labels =  [[0.]
 [1.]
 [1.]
 [0.]]


' optimizer\n    -> 학습하는 함수, 즉 역전파 알고리즘을 뭘로 사용할 것인가?\n\n    torch.optim.SGD(model.parameters(), lr=1)\n    - SGD : 위 optimizer는 gradient descent 방법을 사용하는데, 그 중에서도 SGD(Stochastic Gradient Descent) 방법을 사용한다고 보면 됨\n    - model.paramters() : optimizie 할 대상 (위 예시에서는 가중치행렬인 W, 편향인 b가 모델의 파라미터가 된다)\n    - lr : learning rate\n'

In [5]:
# train mode로 작동하도록 설정
model.train()

# 모델 학습
for epoch in range(1001):
    ''' epoch
        모든 data를 한 번 다 읽은 것을 1 epoch으로 말함
        때문에 모든 데이터를 1000번 읽어서 천천히 수렴해나가겠다라는 뜻
    '''

    # optimizer 초기화
    optimizer.zero_grad()

    ''' optimizer.zero_grad()
        - zero_grad() : 역전파를 수행하기 전에 기울기를 0으로 초기화

        원래 pytorch는 기울기를 누적하는 방식으로 작동함
        -> 기울기를 새로 계산할 때마다 이전에 계산된 기울기 값에 더해지는 방식 ... 미니 배치나 여러 배치를 사용하여 손실을 계산할 때 유용함

        -> RNN 학습 때는 zero_grad를 하지 않음

        하지만! 기본적으로 매 학습 단계마다 기울기를 새로 계산하고, 그 이전 기울기는 사용하지 않음. 때문에 초기화한다.
    '''

    # 모델 설정 후
    hypothesis = model(input_features)

    '''
        Hypothesis는 그냥 model이라고 생각하면 편하겠다.
    '''

    # cost를 계산하고
    cost = loss_func(hypothesis, labels)

    # 역전파 : 손실함수로부터 기울기를 '계산' -> 파라미터 업데이트는 아직 이루어지지 않음!
    cost.backward()

    '''
        참고로 cost.backward()를 통해 기울기를 구하면, 이 값은 파라미터의 .grad로 저장됨
    '''

    # optimizer 작동시켜 역전파를 실행함 -> 파라미터 업데이트
    optimizer.step()

    if epoch%100 == 0:
        print(epoch, cost.item())

    ''' cost.item을 통해 cost값을 불러올 수 있음
    '''

0 0.7235664129257202
100 0.6926396489143372
200 0.6908921003341675
300 0.6803392767906189
400 0.6224880218505859
500 0.5384353399276733
600 0.43271487951278687
700 0.2291935682296753
800 0.12024886906147003
900 0.0768842101097107
1000 0.055503424257040024


In [6]:
# 평가 모드 설정
model.eval()

with torch.no_grad():

    ''' torch.no_grad()
        학습이 아닌 평가를 하는 것이기 때문에 gradient 적용 필요 X
    '''

    hypothesis = model(input_features)
    logits = (hypothesis > 0.5).float()
    predicts = tensor2list(logits)
    golds = tensor2list(labels)

    print("pred = ", predicts)
    print("gold = ", golds)
    print("Accuracy = {0:f}".format(accuracy_score(golds, predicts)))

pred =  [[0.0], [1.0], [1.0], [0.0]]
gold =  [[0.0], [1.0], [1.0], [0.0]]
Accuracy = 1.000000



---

# Wide ANN

hidden layer를 2\*2에서 2\*10으로 변경  
**Widening은 선의 개수를 늘리는 효과**  
→ 학습시간은 더 걸리더라도 안정적인 성능을 낼 수 있음

In [7]:
model = nn.Sequential(
    nn.Linear(2, 10, bias=True), nn.Sigmoid(),
    nn.Linear(2, 1, bias=True), nn.Sigmoid()).to(device)

---

# Shallow ANN

Hidden layer를 없애고 Single-layer Perceptron으로 변경  
single layer perceptron은 linear separable problem만을 풀 수 있기 때문에 non-linear separable problem은 풀 수 없음  
→ 성능 떨어짐

In [8]:
model = nn.Sequential(
    nn.Linear(2 , 1, bias=True), nn.Sigmoid()).to(device)

---

# Deep ANN

hidden layer 층을 1개에서 2개로 변경  
**deeping은 선을 구부리는 효과**

In [9]:
model = nn.Sequential(
    nn.Linear(2, 2, bias=True), nn.Sigmoid(),
    nn.Linear(2, 2, bias=True), nn.Sigmoid(),
    nn.Linear(2, 1, bias=True), nn.Sigmoid()).to(device)

---

# Deeper & Wider ANN

hidden layer 층을 1개에서 7개로 변경  
Q. 과연 성능이 괜찮을까?  
A. 아니다! **Vanishing Gradient** 문제  
→ 활성함수를 sigmoid에서 ReLU를 사용함으로써 해결됨

In [10]:
# sigmoid as activation function
model_org = nn.Sequential(
    nn.Linear(2, 10, bias=True), nn.Sigmoid(),
    nn.Linear(10, 10, bias=True), nn.Sigmoid(),
    nn.Linear(10, 10, bias=True), nn.Sigmoid(),
    nn.Linear(10, 10, bias=True), nn.Sigmoid(),
    nn.Linear(10, 10, bias=True), nn.Sigmoid(),
    nn.Linear(10, 10, bias=True), nn.Sigmoid(),
    nn.Linear(10, 10, bias=True), nn.Sigmoid(),
    nn.Linear(10, 1, bias=True), nn.Sigmoid()).to(device)

# ReLU as activation function

''' 마지막 층이 nn.Sigmoid()인 이유
    : classification 문제를 풀고 있기 때문에 마지막 output은 0과 1 사이의 값으로 바꾸기 위함
'''
model_new = nn.Sequential(
    nn.Linear(2, 10, bias=True), nn.ReLU(),
    nn.Linear(10, 10, bias=True), nn.ReLU(),
    nn.Linear(10, 10, bias=True), nn.ReLU(),
    nn.Linear(10, 10, bias=True), nn.ReLU(),
    nn.Linear(10, 10, bias=True), nn.ReLU(),
    nn.Linear(10, 10, bias=True), nn.ReLU(),
    nn.Linear(10, 10, bias=True), nn.ReLU(),
    nn.Linear(10, 1, bias=True), nn.Sigmoid()).to(device)

---

# Dropout

**학습 과정**중에 지정된 비율로 임의의 연결을 끊음으로써 일반화 성능을 개선하는 방법

In [11]:
model = nn.Sequential(
    nn.Linear(2, 10, bias=True), nn.ReLU(), nn.Dropout(0.1),
    nn.Linear(10, 10, bias=True), nn.ReLU(), nn.Dropout(0.1),
    nn.Linear(10, 10, bias=True), nn.ReLU(), nn.Dropout(0.1),
    nn.Linear(10, 10, bias=True), nn.ReLU(), nn.Dropout(0.1),
    nn.Linear(10, 10, bias=True), nn.ReLU(), nn.Dropout(0.1),
    nn.Linear(10, 10, bias=True), nn.ReLU(), nn.Dropout(0.1),
    nn.Linear(10, 10, bias=True), nn.ReLU(), nn.Dropout(0.1),
    nn.Linear(10, 1, bias=True), nn.Sigmoid()).to(device)

In [12]:
# 평가 모드 시에는 학습 시에 적용했던 드롭 아웃 여부 등을 비적용
model.eval()
with torch.no_grad():
    hypothesis = model(input_features)